# PHY671: Multiscale Materials Modelling, from Atomistic to Macroscopic Methods

This jupyter notebook contains the post-processing codes used to visualize the simulation and calculate the Poisson ratio

In [ ]:
import numpy as np
import pandas as pd
import time
import scipy
from IPython.display import clear_output
import heapq
import pickle 

import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Read .xyz data

In [ ]:
Natoms = 6240 #6240 #680

df = pd.read_table("dump.xyz", skiprows=2, sep='\s+',
                         names=['atom_type', 'x', 'y', 'z'])
df = df.dropna(subset=["z"])
print(len(df)/Natoms, "Frames")
print(df.head())
df = df[["x","y","z"]] 

x0   = df["x"].values.astype(np.float64).reshape(int(len(df)/Natoms),Natoms) 
y0   = df["y"].values.astype(np.float64).reshape(int(len(df)/Natoms),Natoms) 
z0   = df["z"].values.astype(np.float64).reshape(int(len(df)/Natoms),Natoms) 

## Visualization \& Pre-Processing

Figure 1: Check the z-distribution of atoms in the graphene sheet at the last iteration:

In [ ]:
fig = plt.figure()
ax  = plt.gca()
plt.plot(x0[-1,:], z0[-1,:], 'b.')
# plt.axis("square") 
plt.xlim(-10, 140)
plt.ylim(-3, 3)
plt.title("Z-position of atoms at the last simmulation run\n", fontsize=20)
plt.xlabel("x (Å)", fontsize=24)
plt.ylabel("z (Å)", fontsize=24)
ax.tick_params(axis='both', labelsize=20)
plt.grid("on")

Figure 2: Generate a .gif animation of the simulation showing the two phases: equilibration (in blue) and deformation (in red)

In [ ]:
def update(f):    
    line.set_data(x0[f,:], y0[f,:])
    if f > 19:
        line.set_color('red') 
    # plt.axis("square") 
    return line,

fig, ax = plt.subplots(figsize=(10, 10))

line, = ax.plot(x0[0,:], y0[0,:], '.b', markersize=6.) 

ani = animation.FuncAnimation(fig, update, frames=len(x0) , interval=1)

# plt.axis("square") 
plt.xlim(-15, 140)  
plt.ylim(-15, 140) 
plt.plot(1e3, 1e3, 'r.', markersize=6.) # I know this isn't the best practice but it works :) 
plt.title("Graphene under Uniaxial Tension from 0 to 3% strain", fontsize=20)
plt.legend(["Finding Equilibrium in NPT", "Tensile Test in N$\sigma_y\sigma_z$T"], fontsize=15, loc="upper right")
plt.xlabel("x (Å)", fontsize=24)
plt.ylabel("y (Å)", fontsize=24)
plt.tight_layout()
ax.tick_params(axis='both', labelsize=20)

ani.save("anim.gif", writer="pillow", fps=10)

plt.show()

Select a sub-domain of study. This allows us to avoid avoid boundary atoms that jump to the other sides of the boundary box given the periodicity. 

In [ ]:
print(np.shape(x0)) 

xmin = 10.0
xmax = 120.0
ymin = 10.0
ymax = 120.0

subdom = [(x0[0,:]<=xmax) & (x0[0,:]>=xmin) & (y0[0,:]<=ymax) & (y0[0,:]>=ymin)][0]

x = x0[:, subdom]
y = y0[:, subdom]
z = z0[:, subdom] 

Figure 3: Visualize the new sub-domain and the entire domain

In [ ]:
frame = 0
npts = Natoms 

plt.figure(dpi=120)
plt.plot(x0[frame,0:npts], y0[frame,0:npts], '.b', label="Entire Domain")  
plt.plot(x[frame,0:npts], y[frame,0:npts], '.r', label="Studied Subdomain") 
# plt.xlim(-15, 145)  
# plt.ylim(-15, 145)  
plt.axis("square")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1), fontsize=15)  # Legend outside
plt.title("Initial Configuration", fontsize=24)
plt.xlabel("x (Å)", fontsize=24)
plt.ylabel("y (Å)", fontsize=24)
ax = plt.gca() 
ax.tick_params(axis='both', labelsize=20) 
plt.savefig("dom.png") 

Figure 4: Generate a .gif animation of the domain of interest showing the two phases of the simulation: equilibration (in blue) and deformation (in red)

In [ ]:
def update(f):    
    line.set_data(x[f,:], y[f,:])
    if f > 19:
        line.set_color('red') 
    return line,

fig, ax = plt.subplots(figsize=(8, 6))

line, = ax.plot(x[0,:], y[0,:], '.b', markersize=2.) 

ani = animation.FuncAnimation(fig, update, frames=len(x) , interval=1)

# plt.axis("square") 
plt.xlim(xmin-10, xmax+10)  
plt.ylim(ymin-10, ymax+10)  
plt.plot(1e3, 1e3, 'r.', markersize=6.) # I know this isn't the best practice but it works :) 
plt.title("Graphene under Uniaxial Tension from 0 to 3% strain (Domain of Interest)", fontsize=20)
plt.legend(["Finding Equilibrium in NPT", "Tensile Test in N$\sigma_y\sigma_z$T"], fontsize=15, loc="upper right")
plt.xlabel("x (Å)", fontsize=24)
plt.ylabel("y (Å)", fontsize=24)
plt.tight_layout()
ax.tick_params(axis='both', labelsize=20)

ani.save("anim-subdom.gif", writer="pillow", fps=10) 

Figure 5: Show animation live in Jupyter (slow)

In [ ]:
nstart = 20
nend   = 30 #np.shape(x)[0] 

for i in range(nstart, nend):
    clear_output(wait=True) 

    plt.plot(x[i,:], y[i,:], '.r')  
    # plt.xlim(-15, 135) 
    # plt.ylim(-15, 135)  
    plt.axis("square")
    plt.title("Step: " + str(i+1))
    plt.xlabel("X-axis", fontsize=24)
    plt.ylabel("Y-axis", fontsize=24) 
    
    plt.pause(0.001)

plt.show()

## Poisson ratio calculation using global strains

In [ ]:
f_ref = 20
act_frames = len(x)-f_ref
n_pts = 3

GblE  = np.zeros((act_frames, 2, 2), dtype=np.float64)
Gblnu = np.zeros((act_frames, 1   ), dtype=np.float64)

for f in range(act_frames):
    a = np.mean(heapq.nlargest(1, x[f_ref  ,:])) - np.mean(heapq.nsmallest(1, x[f_ref  ,:])) 
    b = np.mean(heapq.nlargest(1, x[f+f_ref,:])) - np.mean(heapq.nsmallest(1, x[f+f_ref,:])) 
    epsx = (b-a)/a
    a = np.mean(heapq.nlargest(n_pts, y[f_ref  ,:])) - np.mean(heapq.nsmallest(n_pts, y[f_ref  ,:])) 
    b = np.mean(heapq.nlargest(n_pts, y[f+f_ref,:])) - np.mean(heapq.nsmallest(n_pts, y[f+f_ref,:])) 
    epsy = (b-a)/a
    
    GblE[f,0,0] = epsx
    GblE[f,1,1] = epsy 
    Gblnu[f]    = - epsy / (epsx+1e-20)

In [ ]:
f_nu  = len(x)-1

a=(max(x[f_ref,:])-min(x[f_ref,:]))
b=(max(x[f_nu,: ])-min(x[f_nu,:]))
epsx = (b-a)/a
print(a,b,epsx) 

a=(max(y[f_ref,:])-min(y[f_ref,:]))
b=(max(y[f_nu,: ])-min(y[f_nu,:]))
epsy1 = (b-a)/a
print(a,b,epsy1) 

a = np.mean(heapq.nlargest(n_pts, y[f_ref,:])) - np.mean(heapq.nsmallest(n_pts, y[f_ref,:])) 
b = np.mean(heapq.nlargest(n_pts, y[f_nu ,:])) - np.mean(heapq.nsmallest(n_pts, y[f_nu ,:])) 
epsy2 = (b-a)/a 
print(a,b,epsy2) 

nu1 = - epsy1/epsx 
nu2 = - epsy2/epsx 

print(nu1, "vs", nu2) 

## Poisson ratio calculation using local strains

In [ ]:
def largest_triangle_side(t):
    # Calculate side lengths using the distance formula
    side_a = np.sqrt((t[1,0] - t[0,0])**2 + (t[1,1] - t[0,1])**2)  # Between (x1, y1) and (x2, y2)
    side_b = np.sqrt((t[2,0] - t[1,0])**2 + (t[2,1] - t[1,1])**2)  # Between (x2, y2) and (x3, y3)
    side_c = np.sqrt((t[0,0] - t[2,0])**2 + (t[0,1] - t[2,1])**2)  # Between (x3, y3) and (x1, y1)
    
    # Return the largest side
    return max(side_a, side_b, side_c)
    
def triangle_area(t):
    return 0.5 * abs(t[0,0] * (t[1,1] - t[2,1]) + t[1,0] * (t[2,1] - t[0,1]) + t[2,0] * (t[0,1] - t[1,1]))

def is_point_inside_triangle(p, tri):
    """Check if point p (x, y) is inside triangle tri [(x1, y1), (x2, y2), (x3, y3)] using barycentric coordinates."""
    x, y = p
    x1, y1, x2, y2, x3, y3 = tri.flatten()

    # Compute barycentric coordinates
    detT = (x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1)
    alpha = ((x2 - x) * (y3 - y) - (x3 - x) * (y2 - y)) / detT
    beta = ((x3 - x) * (y1 - y) - (x1 - x) * (y3 - y)) / detT
    gamma = 1 - alpha - beta
    
    return (0 <= alpha <= 1) and (0 <= beta <= 1) and (0 <= gamma <= 1)

def ensure_outward_normal(midpoint, normal, centroid):
    """ Flips normal if it points towards the centroid """
    to_centroid = centroid - midpoint
    dot_product = np.dot(to_centroid, normal)
    return normal if dot_product < 0 else -normal 

def compute_unit_vectors(t):
    
    x1, y1, x2, y2, x3, y3 = t.flatten()
    
    # Define triangle edges as vectors
    edge1 = np.array([x2 - x1, y2 - y1]) 
    edge2 = np.array([x3 - x2, y3 - y2]) 
    edge3 = np.array([x1 - x3, y1 - y3]) 
    
    # DO NOT Normalize edges to get unit vectors
    unit1 = edge1 #/ np.linalg.norm(edge1)
    unit2 = edge2 #/ np.linalg.norm(edge2)
    unit3 = edge3 #/ np.linalg.norm(edge3)

    # Compute normal vectors 
    normal1 = np.array([-unit1[1], unit1[0]]) 
    normal2 = np.array([-unit2[1], unit2[0]])
    normal3 = np.array([-unit3[1], unit3[0]])
    
    # Compute centroid of the triangle
    centroid = np.array([(x1 + x2 + x3) / 3, (y1 + y2 + y3) / 3])

    # Compute midpoints of edges
    mid1 = np.array([(x1 + x2) / 2, (y1 + y2) / 2])
    mid2 = np.array([(x2 + x3) / 2, (y2 + y3) / 2])
    mid3 = np.array([(x3 + x1) / 2, (y3 + y1) / 2])

    # Adjust normals to ensure outward direction
    normal1 = ensure_outward_normal(mid1, normal1, centroid)
    normal2 = ensure_outward_normal(mid2, normal2, centroid)
    normal3 = ensure_outward_normal(mid3, normal3, centroid)    
    
    # return np.array([unit1, unit2, unit3, normal1, normal2, normal3, mid1, mid2, mid3])
    return np.array([[unit1, unit2, unit3], [normal1, normal2, normal3], [mid1, mid2, mid3]])

In [ ]:
points = np.column_stack((x[f_ref,:],y[f_ref,:]))

print(np.shape(points))

plt.figure(dpi=120, figsize=(8,8))
tri = scipy.spatial.Delaunay(points)  # Generate a triangulation (mesh) of the points
plt.triplot(points[:, 0], points[:, 1], tri.simplices, color='blue')
plt.scatter(points[:, 0], points[:, 1], color='red', s=6.)
plt.xlim(40, 80)  
plt.ylim(40, 80)  
# plt.axis("square") 
plt.title("Delaunay Triangulation (Zoomed into [40,80]x[40,80])\n", fontsize=20)
plt.xlabel("x (Å)", fontsize=24)
plt.ylabel("y (Å)", fontsize=24)
ax = plt.gca()
ax.tick_params(axis='both', labelsize=20)

# plt.savefig("2bis.png")  # Saves as a PNG file
plt.show() 

max_area_allowed = 1.3 * 3 * 2*(1.42*np.sqrt(3)/2) * 1.42/2 
max_len_allowed  = 1.3 * 2 * 1.42*np.sqrt(3)/2

accepted_simplices = np.zeros(tri.simplices.shape[0], dtype=bool)

for i, simplex in enumerate(tri.simplices):  # Iterate through all triangles
    triangle = points[simplex]  # Get triangle vertices

    if True: #triangle_area(triangle) <= max_area_allowed and largest_triangle_side(triangle) <= max_len_allowed: 
        accepted_simplices[i] = True 

e_0    = np.zeros((len(accepted_simplices),3,2), dtype=np.float64) 
e_s_0  = np.zeros((len(accepted_simplices),3,2), dtype=np.float64) 
area_0 = np.zeros((len(accepted_simplices),1 ), dtype=np.float64) 

for i, simplex in enumerate(tri.simplices[accepted_simplices]): 
    triangle = points[simplex] 

    vec = compute_unit_vectors(triangle)

    e_0[i,:,:]   = vec[0] 
    e_s_0[i,:,:] = vec[1] 
    midpt_0      = vec[2] 
    area_0[i]    = triangle_area(triangle)

In [ ]:
E    = np.zeros((act_frames, len(accepted_simplices), 2, 2), dtype=np.float64)
AvgE = np.zeros((act_frames, 2, 2), dtype=np.float64)

t0 = time.time()
for k in np.arange(0,act_frames):

    clear_output(wait=True)  # Clears the previous output in the notebook
    print(str(k+1)+"/"+str(act_frames))
    
    points = np.column_stack((x[k+f_ref,:],y[k+f_ref,:]))
    
    for i, simplex in enumerate(tri.simplices[accepted_simplices]): 
        triangle = points[simplex]  
        
        vec = compute_unit_vectors(triangle) 

        e_i_0   = e_0[i,:,:]
        e_i_s_0 = e_s_0[i,:,:]
        
        e_i_t   = vec[0]
        e_i_s_t = vec[1]
        midpt_0 = vec[2] 
        
        Q = np.zeros((3,1), dtype=np.float64) 
        for j in range(3):
            Q[j] = 0.5 * ( np.dot(e_i_t[j,:], e_i_t[j,:]) - np.dot(e_i_0[j,:], e_i_0[j,:]) )
        
        E[k,i,:,:] = -1./8./area_0[i] * ( (Q[0]-Q[1]-Q[2]) * np.tensordot(e_i_s_0[0,:], e_i_s_0[0,:], axes=0) 
                                        + (Q[1]-Q[0]-Q[2]) * np.tensordot(e_i_s_0[1,:], e_i_s_0[1,:], axes=0) 
                                        + (Q[2]-Q[0]-Q[1]) * np.tensordot(e_i_s_0[2,:], e_i_s_0[2,:], axes=0)
                                        ) 

tf = time.time()
print("It took", tf-t0, "s") 

In [ ]:
# print(np.shape(E),np.shape(area_0))
total_area = np.sum(area_0)
# print(total_area, min(x[0,:]), max(x[0,:]), (max(x[0,:])-min(x[0,:]))*(max(y[0,:])-min(y[0,:])) )

for k in range(np.shape(E)[0]):
    for i in range(2):
        for j in range(2):
            AvgE[k,i,j] = np.sum(E[k,:,i,j]*area_0[:, 0])/total_area

In [ ]:
## 1
plt.figure(dpi=120) #dpi=120

max_strain = 0.03

for f in range(act_frames):
    a=(max(x[f_ref,:])-min(x[f_ref,:]))
    b=(max(x[f+f_ref,:])-min(x[f+f_ref,:]))
    
    plt.plot((b-a)/a, AvgE[f,0,0], 'bo', markersize=6)
    plt.plot([0,max_strain], [0, max_strain], 'r') 
plt.xlabel(r'Global $\varepsilon _x$ (Å/Å)', fontsize=24) 
plt.ylabel(r'Avg. Local $<\varepsilon _x>$ (Å/Å)', fontsize=24) 
plt.title(r"Calculated Strain vs. Imposed Strain", fontsize=20)
plt.legend(["Calculated Local Strains", "Perfect fit"], loc="best", fontsize=20, bbox_to_anchor= (1, -0.3))
plt.grid() 
ax = plt.gca()
ax.tick_params(axis='both', labelsize=20)
plt.savefig("calc-strain.png") 


## 2
plt.figure(dpi=120)

for f in range(act_frames):    
    n_pts = 3
    a = np.mean(heapq.nlargest(n_pts, y[f_ref  ,:])) - np.mean(heapq.nsmallest(n_pts, y[f_ref  ,:])) 
    b = np.mean(heapq.nlargest(n_pts, y[f+f_ref,:])) - np.mean(heapq.nsmallest(n_pts, y[f+f_ref,:])) 
    epsy = (b-a)/a
    
    plt.plot(epsy, AvgE[f,1,1], 'bo', markersize=6)
plt.xlabel(r'Global $\varepsilon _y$ (Å/Å)', fontsize=24) 
plt.ylabel(r'Avg. Local $<\varepsilon> _y$ (Å/Å)', fontsize=24) 
plt.title(r"Calculated Strain vs. Global Strain", fontsize=20)
plt.grid()
ax = plt.gca()
ax.tick_params(axis='both', labelsize=20)
plt.savefig("calc-strain-y.png") 


## 3
plt.figure(dpi=120)

plt.plot(AvgE[:,0,0], AvgE[:,1,1], 'bo', markersize=6)
plt.xlabel(r'Avg. Local $<\varepsilon _x>$ (Å/Å)', fontsize=24) 
plt.ylabel(r'Avg. Local $<\varepsilon _y>$ (Å/Å)', fontsize=24) 
plt.grid()
ax = plt.gca()
ax.tick_params(axis='both', labelsize=20)


## 4
plt.figure(dpi=120)

for f in range(act_frames):
    n_pts = 3
    a = np.mean(heapq.nlargest(n_pts, x[f_ref  ,:])) - np.mean(heapq.nsmallest(n_pts, x[f_ref  ,:])) 
    b = np.mean(heapq.nlargest(n_pts, x[f+f_ref,:])) - np.mean(heapq.nsmallest(n_pts, x[f+f_ref,:])) 
    epsx = (b-a)/a

    nu = -AvgE[f,1,1]/(AvgE[f,0,0]+1e-20)
    
    if nu > 0.5 or nu < -1:
        continue 
    
    plt.plot(epsx, nu, 'bo', markersize=6)
plt.xlabel(r'Global $\varepsilon _x$ (Å/Å)', fontsize=24) # r'Avg. Local $<\varepsilon _x>$ (Å/Å)'
plt.ylabel(r"$<\nu>=-<\varepsilon _y>/<\varepsilon _x>$ (-)", fontsize=20) 
plt.grid() 
ax = plt.gca()
ax.tick_params(axis='both', labelsize=20)

plt.figure(dpi=120) 

for f in range(act_frames):
    
    f = f + f_ref  # We don t use AvgE here so I can do this
    a = np.mean(heapq.nlargest(1, x[f_ref,:])) - np.mean(heapq.nsmallest(1, x[f_ref,:])) 
    b = np.mean(heapq.nlargest(1, x[f    ,:])) - np.mean(heapq.nsmallest(1, x[f    ,:])) 
    epsx = (b-a)/a
    
    n_pts = 3
    a = np.mean(heapq.nlargest(n_pts, y[f_ref,:])) - np.mean(heapq.nsmallest(n_pts, y[f_ref,:])) 
    b = np.mean(heapq.nlargest(n_pts, y[f    ,:])) - np.mean(heapq.nsmallest(n_pts, y[f    ,:])) 
    epsy = (b-a)/a
    
    plt.plot(epsx, epsy, 'ro--')
plt.xlabel(r'Global $\varepsilon _x$ (Å/Å)', fontsize=24) 
plt.ylabel(r'Global $\varepsilon _y$ (Å/Å)', fontsize=24) 
plt.grid()
ax = plt.gca()
ax.tick_params(axis='both', labelsize=20) 


## 5
plt.figure(dpi=120)

for f in range(act_frames):

    f = f + f_ref  # We don t use AvgE here so I can do this
    a = np.mean(heapq.nlargest(1, x[f_ref,:])) - np.mean(heapq.nsmallest(1, x[f_ref,:])) 
    b = np.mean(heapq.nlargest(1, x[f    ,:])) - np.mean(heapq.nsmallest(1, x[f    ,:])) 
    epsx = (b-a)/a

    n_pts = 3
    a = np.mean(heapq.nlargest(n_pts, y[f_ref,:])) - np.mean(heapq.nsmallest(n_pts, y[f_ref,:])) 
    b = np.mean(heapq.nlargest(n_pts, y[f    ,:])) - np.mean(heapq.nsmallest(n_pts, y[f    ,:])) 
    epsy = (b-a)/a

    nu = - epsy / (epsx+1e-20)

    if nu > 0.35 or nu < -1:
        continue
    
    plt.plot(epsx, nu, 'ro--')
plt.title(r"Variation of Poisson's ratio $\nu$ with strain")
plt.xlabel(r'$\varepsilon _x$ (Å/Å)', fontsize=24) 
plt.ylabel(r"Poisson's Ratio $\nu$ (-)", fontsize=24) 
plt.grid("on")
ax = plt.gca()
ax.tick_params(axis='both', labelsize=20)

# plt.savefig("nu-eps.png")

Store values as .pkl

In [ ]:
with open('Lx.pkl', 'wb') as f:
    pickle.dump(AvgE, f)
    pickle.dump(GblE, f) 

## Compare multiple simulations

In [ ]:
n_frames = 51

GE = np.zeros((4, n_frames, 2, 2), dtype=np.float64)
AE = np.zeros((4, n_frames, 2, 2), dtype=np.float64)

for i in range(4):
    with open("L"+str(i+1)+".pkl", 'rb') as f: 
        AE[i, :, :, :] = pickle.load(f)[:51, :, :]
        GE[i, :, :, :] = pickle.load(f)[:51, :, :]

In [ ]:
plt.figure(dpi=120)

# print(np.shape(y), n_frames) 

color = ['b', 'r', 'g', 'm', 'c', 'k', 'y']

for i in range(4):

    epsx = GE[i,:,0,0]

    nu = np.zeros(len(epsx))
    for f in range(n_frames):
        nu[f] = -AE[i,f,1,1]/(AE[i,f,0,0]+1e-20)
        
        # if nu[f] > 0.5 or nu[f] < -1:
        #     nu[f] = 0 # continue
    
    plt.plot(epsx, nu, 'o', color=color[i], markersize=6)
        # plt.plot([0,-max_strain/5], [0, -max_strain/5], 'r') 
plt.xlabel(r'Global $\varepsilon _x$ (Å/Å)', fontsize=24) # r'Avg. Local $<\varepsilon _x>$ (Å/Å)'
plt.ylabel(r"$<\nu>=-<\varepsilon _y>/<\varepsilon _x>$ (-)", fontsize=20) 
plt.ylim([-1., 0.3])
plt.legend(["Tersoff, 300K", "LCBOP, 300K", "Tersoff, 1000K", "Tersoff, 100K"], fontsize=15, loc="lower right") # , bbox_to_anchor=(1, -0.4)
plt.title("Averaged local Poisson's ratio versus Applied Strain \n for Two Different Potentials and Three Finite Temperatures \n", fontsize=20)
plt.grid() 
ax = plt.gca()
ax.tick_params(axis='both', labelsize=20)
plt.savefig("local-results.png")

In [ ]:
plt.figure(dpi=120)

color = ['b', 'm', 'r', 'g']

for i in range(4):

    epsx = GE[i,:,0,0]

    nu = np.zeros(len(epsx))
    for f in range(n_frames):
        nu[f] = -GE[i,f,1,1]/(GE[i,f,0,0]+1e-20)
        
        # if nu[f] > 0.5 or nu[f] < -1:
        #     nu[f] = 0 # continue
    
    plt.plot(epsx, nu, 'o', color=color[i], markersize=6)
        # plt.plot([0,-max_strain/5], [0, -max_strain/5], 'r') 
plt.xlabel(r'Global $\varepsilon _x$ (Å/Å)', fontsize=24) # r'Avg. Local $<\varepsilon _x>$ (Å/Å)'
plt.ylabel(r"Global $\nu=-\varepsilon _y/\varepsilon _x$ (-)", fontsize=20) 
plt.ylim([-0.35, 0.5])
plt.legend(["Tersoff, 300K", "LCBOP, 300K", "Tersoff, 1000K", "Tersoff, 100K"], fontsize=15, loc="lower right") # , bbox_to_anchor=(1, -0.4)
plt.title("Global Poisson's ratio versus Applied Strain \n for Two Different Potentials and Three Finite Temperatures \n", fontsize=20)
plt.grid() 
ax = plt.gca()
ax.tick_params(axis='both', labelsize=20)
plt.savefig("global-results.png")

Bonus: Fit local strain Poisson ratio curves to $<\nu>=c_1 \tanh (x) + c_2$ (not working)

In [ ]:
# def model(x, c1, c2):
#     return c1 * np.tanh(x) + c2

# epsx = GE[0,40:,0,0]
# i = 0 
# nu = np.zeros(len(epsx))
# for f in range(n_frames-40):
#     nu[f] = -AE[i,f,1,1]/(AE[i,f,0,0]+1e-20)
    
# popt, pcov = scipy.optimize.curve_fit(model, epsx, nu, p0=[1, 0])  # Initial guess for [c1, c2]

# c1_fit, c2_fit = popt
# print(f"Fitted Parameters: c1 = {c1_fit:.3f}, c2 = {c2_fit:.3f}")

# plt.scatter(epsx, nu, label="Data", color="red")
# plt.plot(epsx, model(epsx, *popt), label="Fitted Curve", color="blue")
# plt.xlabel(r"$\varepsilon_x$")
# plt.ylabel(r"$<\nu>$")
# plt.ylim([-1., 1.3])
# plt.legend()
# plt.title(r"Nonlinear Fit: y = c1 * tanh($\varepsilon_x$) + c2")
# plt.show() 